In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import utils as vutils
import numpy as np
import os
from pathlib import Path

# our implement
from model import Net
from myDataset import MyDataset

In [2]:

weights_root = Path("weights")
weights_root.mkdir(exist_ok=True)

transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])
data_list = ['normal', 'bad_leg']
mydataset = MyDataset(data_dir=data_list, transforms=transforms_)
train_len = int(len(mydataset)*0.8)
test_len = len(mydataset) - train_len
train_set, val_set = torch.utils.data.random_split(mydataset, [train_len, test_len])

AttributeError: 'Compose' object has no attribute 'T'

In [ ]:
len(train_set)

6132

In [ ]:
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))




net = Net()
net.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.002)
criterion  = nn.CrossEntropyLoss()


transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])

# prepare data    
data_list = ['normal', 'bad_leg']
mydataset = MyDataset(data_list, transforms_)
train_len = int(len(mydataset)*0.8)
test_len = len(mydataset) - train_len
train_set, test_set = torch.utils.data.random_split(mydataset, [train_len, test_len])


train_loader = DataLoader(
    train_set,
    batch_size=64,
    shuffle=True,
    pin_memory=True
)
test_loader = DataLoader(
    test_set,
    batch_size=64,
    shuffle=True,
    pin_memory=True
)


device 0 NVIDIA GeForce RTX 3080 Laptop GPU
normal
bad_leg
(3, 25)
(7666, 3, 25)


In [ ]:
print(test_set[0][0][0].shape)

torch.Size([3, 25])


In [ ]:
# test_net = Net()
# test_net.to(device)

# data = test_set[0][0].to(device)
# test_net.load_state_dict(torch.load(PATH))
# output = test_net(data)
# output

In [ ]:
training_start = time.time()
for epoch in range(1,50):
    loss_all = 0
    net.train()
    for i, batch in enumerate(train_loader):
        
        
        data = batch[0].to(device)
        label = batch[1].to(device)
        optimizer.zero_grad()
        output = net(data).squeeze()
        loss = criterion(output, label.long())
        loss.backward()
        loss_all += loss.item()
        optimizer.step()
        
        print(f'\rloss: {loss.item()}',end='')
        # print('\r',loss_all/(i+epoch*len(dataloader)))
        

    acc = 0
    for i, batch in enumerate(test_loader):
        
        net.eval()
        data = batch[0].to(device)
        label = batch[1].to(device)
        
        output = net(data).squeeze()
        #output = output.topk(1)[1].squeeze()
        acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
        
    print(' acc: ',acc/(i+1))
    torch.save(net.state_dict(), weights_root / f"E{epoch+1:03}.pth")
        
    
        
        

       


loss: 0.1205064058303833 acc:  0.9563382056451614
loss: 0.127134621143341064 acc:  0.9758904569892474
loss: 0.045047149062156686 acc:  0.9778435819892474
loss: 0.021637272089719772 acc:  0.9850260416666666
loss: 0.0064468635246157655 acc:  0.9700520833333334
loss: 0.0600216723978519445 acc:  0.9928385416666666
loss: 0.0049793613143265251 acc:  0.9895623319892474
loss: 0.0053337360732257374 acc:  0.9947916666666666
loss: 0.0267143584787845657 acc:  0.9927965389784946
loss: 0.0265705306082963946 acc:  0.9928385416666666
loss: 0.0361079797148704578 acc:  0.9895203293010754
loss: 0.04901303723454475455 acc:  0.9817708333333334
loss: 0.0016527993138879538 acc:  0.992124495967742
loss: 0.0075131836347281935 acc:  0.9934895833333334
loss: 0.005610331892967224 acc:  0.990192372311828
loss: 0.02219497226178646787 acc:  0.9908854166666666
loss: 0.03904959186911583633 acc:  0.9967237903225806
loss: 0.001034916378557682 acc:  0.99609375
loss: 0.03407261520624161456 acc:  0.9934685819892474
loss: 0

In [ ]:
test_net = Net()

In [ ]:
weight = 'E050.pth'
PATH = os.path.join(weights_root, weight)
test_net = Net()
test_net.to(device)
acc = 0
for i, batch in enumerate(test_loader):
    data = batch[0].to(device)
    label = batch[1].to(device)
    print(data.shape)
    print(data)
    test_net.load_state_dict(torch.load(PATH))
    output = test_net(data)
    acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
    

print(acc/(i+1))
    
    

torch.Size([64, 1, 3, 25])
tensor([[[[-1.8523, -1.7876, -1.9817,  ..., -1.0000, -1.0000, -1.9167],
          [-1.6553, -1.2254, -1.2368,  ..., -1.0000, -1.0000, -0.2785],
          [ 0.8338,  0.8194,  0.6387,  ..., -0.7736, -1.0000, -0.4731]]],


        [[[-1.1918, -1.3769, -1.4576,  ..., -1.0000, -1.0000, -0.8294],
          [-1.4462, -1.0501, -1.0605,  ..., -1.0000, -1.0000,  0.8999],
          [ 0.7305,  0.6461,  0.6497,  ..., -0.7392, -1.0000, -0.0915]]],


        [[[-1.0706, -1.2641, -1.3366,  ..., -1.0000, -1.0000, -0.8128],
          [-1.4022, -1.0499, -1.0607,  ..., -1.0000, -1.0000,  0.8119],
          [ 0.9086,  0.7145,  0.6578,  ..., -0.6361, -0.6499, -0.0862]]],


        ...,


        [[[-1.1271, -1.3204, -1.4094,  ..., -1.0000, -1.0000, -0.8213],
          [-1.4569, -1.0497, -1.0497,  ..., -1.0000, -1.0000,  0.9006],
          [ 0.7193,  0.7288,  0.6593,  ..., -0.7308, -0.6205, -0.1845]]],


        [[[-1.6026, -1.5866, -1.8122,  ..., -1.3207, -1.3368, -1.3285],
      

In [ ]:
acc = 0
for i, batch in enumerate(test_loader):
    
    net.eval()
    data = batch[0].to(device)
    label = batch[1].to(device)
    
    output = net(data).squeeze()
    #print(output)
    #output = output.topk(1)[1].squeeze()
    print(torch.argmax(output, dim=1))
    print(label.int())
    print((torch.argmax(output, dim=1) == label.int()))
    acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
    
print(acc/(i+1))
        

       

tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0], device='cuda:0',
       dtype=torch.int32)
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True], device='cuda:0')
tensor([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 

In [ ]:
a = np.array([1,2,3])
b = np.array([1,1,1])

print((a == b).mean())

0.3333333333333333
